<a href="https://colab.research.google.com/github/amrutadeo-22/resnet/blob/main/Eff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils

# Swish activation function
def swish(x):
    return x * tf.nn.sigmoid(x)

# Squeeze and Excitation block
def squeeze_and_excite(inputs, reduction=4):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling2D()(inputs)
    se = layers.Reshape((1, 1, filters))(se)
    se = layers.Dense(filters // reduction, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    return layers.Multiply()([inputs, se])

# MBConv block
def mb_conv_block(inputs, filters, expansion_factor, kernel_size, strides, reduction=4):
    x = layers.Conv2D(filters * expansion_factor, kernel_size=1, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    x = layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    x = squeeze_and_excite(x, reduction)

    x = layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    if strides == 1 and inputs.shape[-1] == filters:
        x = layers.Add()([x, inputs])

    return x

# EfficientNet model
def efficient_net(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, kernel_size=3, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    x = mb_conv_block(x, filters=16, expansion_factor=1, kernel_size=3, strides=1)

    x = mb_conv_block(x, filters=24, expansion_factor=6, kernel_size=3, strides=2)
    x = mb_conv_block(x, filters=24, expansion_factor=6, kernel_size=3, strides=1)

    x = mb_conv_block(x, filters=40, expansion_factor=6, kernel_size=5, strides=2)
    x = mb_conv_block(x, filters=40, expansion_factor=6, kernel_size=5, strides=1)

    x = mb_conv_block(x, filters=80, expansion_factor=6, kernel_size=3, strides=2)
    x = mb_conv_block(x, filters=80, expansion_factor=6, kernel_size=3, strides=1)

    x = mb_conv_block(x, filters=112, expansion_factor=6, kernel_size=5, strides=1)
    x = mb_conv_block(x, filters=112, expansion_factor=6, kernel_size=5, strides=1)

    x = mb_conv_block(x, filters=192, expansion_factor=6, kernel_size=5, strides=2)
    x = mb_conv_block(x, filters=192, expansion_factor=6, kernel_size=5, strides=1)

    x = mb_conv_block(x, filters=320, expansion_factor=6, kernel_size=3, strides=1)

    x = layers.Conv2D(1280, kernel_size=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, x)
    return model

# Load CIFAR-10 dataset
(input_train, target_train), (input_test, target_test) = datasets.cifar10.load_data()

input_train = input_train.astype('float32') / 255.0
input_test = input_test.astype('float32') / 255.0

target_train = utils.to_categorical(target_train, 10)
target_test = utils.to_categorical(target_test, 10)

# Create model
model = efficient_net(input_shape=(32, 32, 3), num_classes=10)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(input_train, target_train, epochs=25, batch_size=64, validation_data=(input_test, target_test))


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 1069s 1s/step - accuracy: 0.2853 - loss: 1.9542 - val_accuracy: 0.4241 - val_loss: 2.1079
Epoch 2/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 1005s 1s/step - accuracy: 0.4933 - loss: 1.4291 - val_accuracy: 0.3762 - val_loss: 6.2052
Epoch 3/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 1031s 1s/step - accuracy: 0.5430 - loss: 1.2889 - val_accuracy: 0.5412 - val_loss: 1.3599
Epoch 4/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 1043s 1s/step - accuracy: 0.5758 - loss: 1.2323 - val_accuracy: 0.5704 - val_loss: 1.2976
Epoch 5/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 1030s 1s/step - accuracy: 0.6406 - loss: 1.0391 - val_accuracy: 0.6224 - val_loss: 1.1260
Epoch 6/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 1042s 1s/step - accuracy: 0.6762 - loss: 0.9245 - val_accuracy: 0.6327 - val_loss: 1.1321
Epoch 7/25
782/782 ━━━━━━━━━━━━━━━━━━━━ 1030s 1s/step - accuracy: 0.6925 - loss: 0.8833 - val_accuracy: 0.6281 - val_loss: 1.2039
Epoch 8/25
782/782 ━━━━━━━━━━━━━━━━━━